In [24]:
import os
import pickle
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import dataset
from vocab import Vocab
import csv


ModuleNotFoundError: No module named 'vocab'

In [21]:

class ImdbDataset(Dataset):
    def __init__(self, train=True, sequence_max_len=100):
        self.sequence_max_len = sequence_max_len
        self.data_path = r"./stsb-en-train.csv" if train else r"./stsb-en-test.csv"
        self.data_set = []
        # 逐行读取csv文件
        with open(self.data_path, 'r', encoding="utf-8") as csvfile:
            reader = csv.reader(csvfile)
            for header_row in reader:
                self.data_set.append(header_row)
        self.threshold = 3.0
            
    def __getitem__(self, idx):
        # 从self.data_set获取评论并分词
        sentences = self.data_set[idx]
        review1 = dataset_train.tokenlize(sentences[0])
        review2 = dataset_train.tokenlize(sentences[1])
        return review1, review2

    def __len__(self):
        return len(self.data_set)


def collate_fn(batch):
    """
    对batch数据进行处理
    :param batch: [一个getitem的结果,getitem的结果,getitem的结果]
    :return: 元组
    """
    # reviews, labels = zip(*batch)
    return batch


def get_dataloader(train=True):
    imdb_dataset = ImdbDataset(train, sequence_max_len=100)
    my_dataloader = DataLoader(imdb_dataset, batch_size=200, shuffle=True, collate_fn=collate_fn)
    return my_dataloader


if __name__ == '__main__':

    ws = Vocab()
    dl_train = get_dataloader(True)
    dl_test = get_dataloader(False)
    cnt = 1
    for reviews in tqdm(dl_train, total=len(dl_train)):
        # 一个sentence是一对具有一定相似度的语句
        for sentence1, sentence2 in reviews:
            ws.fit(sentence1)
            ws.fit(sentence2)
    

    for reviews in tqdm(dl_test, total=len(dl_test)):
        for sentence1, sentence2 in reviews:
            ws.fit(sentence1)
            ws.fit(sentence2)

    ws.build_vocab()
    print(len(ws))
    if not os.path.exists("./models"):
        os.makedirs("./models")
    pickle.dump(ws, open("./models/vocab.pkl", "wb"))


ModuleNotFoundError: No module named 'dataset_train'

In [ ]:
import os
import pickle
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import dataset_train
from vocab import Vocab
import csv

class ImdbDataset(Dataset):
    def __init__(self, train=True, sequence_max_len=100):
        self.sequence_max_len = sequence_max_len
        self.data_path = r"./stsb-en-train.csv" if train else r"./stsb-en-test.csv"
        self.data_set = []
        # 逐行读取csv文件
        with open(self.data_path, 'r', encoding="utf-8") as csvfile:
            reader = csv.reader(csvfile)
            for header_row in reader:
                self.data_set.append(header_row)
        self.threshold = 3.0
            
    def __getitem__(self, idx):
        # 从self.data_set获取评论并分词
        sentences = self.data_set[idx]
        review1 = dataset_train.tokenlize(sentences[0])
        review2 = dataset_train.tokenlize(sentences[1])
        return review1, review2

    def __len__(self):
        return len(self.data_set)


def collate_fn(batch):
    """
    对batch数据进行处理
    :param batch: [一个getitem的结果,getitem的结果,getitem的结果]
    :return: 元组
    """
    # reviews, labels = zip(*batch)
    return batch


def get_dataloader(train=True):
    imdb_dataset = ImdbDataset(train, sequence_max_len=100)
    my_dataloader = DataLoader(imdb_dataset, batch_size=200, shuffle=True, collate_fn=collate_fn)
    return my_dataloader


if __name__ == '__main__':

    ws = Vocab()
    dl_train = get_dataloader(True)
    dl_test = get_dataloader(False)
    cnt = 1
    for reviews in tqdm(dl_train, total=len(dl_train)):
        # 一个sentence是一对具有一定相似度的语句
        for sentence1, sentence2 in reviews:
            ws.fit(sentence1)
            ws.fit(sentence2)
    

    for reviews in tqdm(dl_test, total=len(dl_test)):
        for sentence1, sentence2 in reviews:
            ws.fit(sentence1)
            ws.fit(sentence2)

    ws.build_vocab()
    print(len(ws))
    if not os.path.exists("./models"):
        os.makedirs("./models")
    pickle.dump(ws, open("./models/vocab.pkl", "wb"))


In [ ]:
import os
import re
import zipfile
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import pickle
from vocab import Vocab
import csv


In [ ]:

Vocab()

class ImdbDataset(Dataset):
    def __init__(self, train=True, sequence_max_len=100):
        self.sequence_max_len = sequence_max_len
        self.data_path = r"./stsb-en-train.csv" if train else r"./stsb-en-test.csv"
        self.data_set = []
        # 逐行读取csv文件
        with open(self.data_path, 'r', encoding="utf-8") as csvfile:
            reader = csv.reader(csvfile)
            for header_row in reader:
                self.data_set.append(header_row)
        self.threshold = 3.0

    def __getitem__(self, idx):
        # 从self.data_set获取评论并分词
        sentences = self.data_set[idx]
        review1 = tokenlize(sentences[0])
        review2 = tokenlize(sentences[1])
        # 获取句子的相似度score
        score = float(sentences[2])
        label = 1 if score >= self.threshold else 0
        return review1, review2, label

    def __len__(self):
        return len(self.data_set)


def tokenlize(sentence):
    """
    进行文本分词
    :param sentence: str
    :return: [str,str,str]
    """
    fileters = ['!', '"', '#', '$', '%', '&', '\(', '\)', '\*', '\+', ',', '-', '\.', '/', ':', ';', '<', '=', '>',
                '\?', '@', '\[', '\\', '\]', '^', '_', '`', '\{', '\|', '\}', '~', '\t', '\n', '\x97', '\x96', '”',
                '“', ]
    sentence = sentence.lower()  # 把大写转化为小写
    sentence = re.sub("<br />", " ", sentence)
    sentence = re.sub("|".join(fileters), " ", sentence)
    result = [i for i in sentence.split(" ") if len(i) > 0]

    return result


# 以下为调试代码
def collate_fn(batch):
    """
    对batch数据进行处理
    :param batch: [一个getitem的结果,getitem的结果,getitem的结果]
    :return: 元组
    """
    reviews1, reviews2, labels = zip(*batch)

    return reviews1, reviews2, labels


if __name__ == "__main__":

    imdb_dataset = ImdbDataset(True)
    my_dataloader = DataLoader(imdb_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
    for data in my_dataloader:
        print(data)
        break


In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader
import pickle
from tqdm import tqdm
import dataset_train
from vocab import Vocab
import numpy as np

ModuleNotFoundError: No module named 'dataset_train'

In [ ]:


Vocab()

train_batch_size = 512
test_batch_size = 128
sequence_max_len = 100
embedding_dim = 200
vocab = pickle.load(open("./models/vocab.pkl", "rb"))


def collate_fn(batch):
    """
    对batch数据进行处理
    :param batch: [一个getitem的结果,getitem的结果,getitem的结果]
    :return: 元组
    """
    reviews1, reviews2, labels = zip(*batch)
    reviews1 = torch.LongTensor([vocab.transform(i, max_len=sequence_max_len) for i in reviews1])
    reviews2 = torch.LongTensor([vocab.transform(i, max_len=sequence_max_len) for i in reviews2])
    labels = torch.LongTensor(labels)
    return reviews1, reviews2, labels

# def get_dataset():
#     return dataset_train.ImdbDataset(train)

def get_dataloader(train=True):
    imdb_dataset = dataset_train.ImdbDataset(train)
    batch_size = train_batch_size if train else test_batch_size
    return DataLoader(imdb_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)


class ImdbModel(nn.Module):
    def __init__(self):
        super(ImdbModel, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=len(vocab), embedding_dim=embedding_dim, padding_idx=vocab.PAD).to()
        self.lstm = nn.LSTM(input_size=200, hidden_size=64, num_layers=2, batch_first=True, bidirectional=True,
                            dropout=0.5)
        self.fc1 = nn.Linear(64 * 2, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, input1, input2):
        """
        :param input:[batch_size,max_len]
        :return:
        """
        input_embeded1 = self.embedding(input1)  # input embeded :[batch_size,max_len,200]
        input_embeded2 = self.embedding(input2)

        output, (h_n1, c_n) = self.lstm(input_embeded1)  # h_n :[4,batch_size,hidden_size]
        output, (h_n2, c_n) = self.lstm(input_embeded2)
        # out :[batch_size,hidden_size*2]
        out1 = torch.cat([h_n1[-1, :, :], h_n1[-2, :, :]], dim=-1)  # 拼接正向最后一个输出和反向最后一个输出
        out2 = torch.cat([h_n2[-1, :, :], h_n2[-2, :, :]], dim=-1)
        # 进行全连接
        out_fc1_1 = self.fc1(out1)
        out_fc1_2 = self.fc1(out2)
        # 进行relu
        out_fc1_relu1 = F.relu(out_fc1_1)
        out_fc1_relu2 = F.relu(out_fc1_2)
        # 全连接
        out_fc2_1 = self.fc2(out_fc1_relu1)  # out :[batch_size,2]
        out_fc2_2 = self.fc2(out_fc1_relu2)
        final_out = torch.cosine_similarity(out_fc2_1, out_fc2_2, dim=1)
        return final_out


def device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')


def train(imdb_model, epoch):
    """
    :param imdb_model:
    :param epoch:
    :return:
    """
    train_dataloader = get_dataloader(train=True)
    optimizer = Adam(imdb_model.parameters())
    criterion = nn.MSELoss()
    for i in range(epoch):
        bar = tqdm(train_dataloader, total=len(train_dataloader))
        for idx, (data1, data2, target) in enumerate(bar):
            optimizer.zero_grad()
            data1 = data1.to(device())
            data2 = data2.to(device())
            target = target.float().to(device())
            output = imdb_model(data1, data2)
            loss = criterion(output, target)
            # loss = F.mse_loss(output, target)
            loss.backward()
            optimizer.step()
            bar.set_description("epcoh:{}  idx:{}   loss:{:.6f}".format(i, idx, loss.item()))
    # 保存模型
    path_model = "./models/lstm_model.pkl"
    torch.save(imdb_model, path_model)
    # 保存模型参数
    path_state_dict = "./models/lstm_model_state_dict.pkl"
    net_state_dict = imdb_model.state_dict()
    torch.save(net_state_dict, path_state_dict)


def calculate_accuracy(predictions, targets, threshold):
    predicted_labels = (predictions > threshold).astype(int)
    true_labels = (targets > threshold).astype(int)
    accuracy = np.mean(predicted_labels == true_labels)
    return accuracy


def test(imdb_model):
    threshold = 0.2
    """
    验证模型
    :param imdb_model:
    :return:
    """
    test_loss = 0
    imdb_model.eval()
    test_dataloader = get_dataloader(train=False)
    predictions = []
    targets = []

    i = 0

    with torch.no_grad():
        for data1, data2, target in tqdm(test_dataloader):
            data1 = data1.to(device())
            data2 = data2.to(device())
            target = target.float().to(device())
            output = imdb_model(data1, data2)
            
            loss = F.mse_loss(output, target)

            test_loss += loss.item()

            # 将 predictions 和 targets 转换成 numpy
            predicted_scores = output.cpu().numpy()
            
            true_scores = target.cpu().numpy()

            predictions.extend(predicted_scores)
            targets.extend(true_scores)

    test_loss /= len(test_dataloader.dataset)
    accuracy = calculate_accuracy(np.array(predictions), np.array(targets), threshold)

    print(f"Test Loss: {test_loss:.4f}")
    print(f"Accuracy: {accuracy:.4f} (Total num: {len(test_dataloader.dataset)} Threshold: {threshold})")


def cal_score(imdb_model):
    predictions = []
    targets= []
    test_dataloader = get_dataloader(train=False)


    with torch.no_grad():
        for data1, data2, target in tqdm(test_dataloader):
            data1 = data1.to(device())
            data2 = data2.to(device())
            target = target.float().to(device())
            output = imdb_model(data1, data2)

            # 将 predictions 和 targets 转换为 numpy
            predicted_scores = output.cpu().numpy()
            true_score = target.cpu().numpy()

            predictions.extend(predicted_scores)
            targets.extend(true_score)
    
    # 计算基准分数(Mean)
    predictions = np.array(predictions)
    targets = np.array(targets)
    target_score = np.mean(targets)
    prediction_score = np.mean(predictions)

    print(f"Mean Score (Baseline): {target_score:.4f}")
    print(f"Mean Score (prediction): {prediction_score:.4f}")

if __name__ == '__main__':
    # 训练
    imdb_model = ImdbModel().to(device())
    train(imdb_model, 10)
    # 测试
    # imdb_dataset = get_dataset()
    path_lstm_model = "./models/lstm_model.pkl"
    lstm_model = torch.load(path_lstm_model)
    test(lstm_model)
    cal_score(lstm_model)
